In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

from generate_model import *
from get_densities import *

In [2]:
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Input, Embedding
from keras.layers import Flatten, Merge, Activation
from keras.layers import Conv2D, Conv3D, MaxPooling2D, MaxPooling3D

### Data Exploration

In [5]:
data_dft = pickle.load(open('atomic_data_clean.p', 'rb'))
atom_data = data_dft[:,:-1]
energy_data = data_dft[:,-1]
atom_data

array([[  8.00000000e+00,  -3.09656111e+02,  -3.12385000e-01, ...,
         -3.15626326e+02,  -1.74244300e+00,   6.21118000e-01],
       [  8.00000000e+00,  -6.92771000e-01,  -5.99800000e-03, ...,
         -6.66298600e+00,  -1.43605600e+00,   2.06315800e+00],
       [  8.00000000e+00,  -4.12569700e+00,  -5.99800000e-03, ...,
         -1.00959120e+01,  -1.43605600e+00,   2.06315800e+00],
       ..., 
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00],
       [  0.00000000e+00,   0.00000000e+00,   0.00000000e+00, ...,
          0.00000000e+00,   0.00000000e+00,   0.00000000e+00]])

In [6]:
energy_data

array([  3.77380000e-08,   1.28601000e-07,   1.60610000e-07, ...,
         0.00000000e+00,   0.00000000e+00,   0.00000000e+00])

In [8]:
data_shape = atom_data.shape

In [15]:
from sklearn.cross_validation import KFold
from sklearn.cross_validation import train_test_split

train_atom, test_atom, train_en, test_en = train_test_split(atom_data, energy_data)

### Building Models

In [20]:
inputs = Input(shape=(data_shape[1],), dtype='float32')

dense_layer0 = Dense(units=80, activation='relu', name='dense0')(inputs)
drop_layer0 = Dropout(rate=0.2, name='drop0')(dense_layer0)

dense_layer1 = Dense(units=50, activation='relu', name='dense1')(drop_layer0)
drop_layer1 = Dropout(rate=0.2, name='drop1')(dense_layer1)

dense_layer2 = Dense(units=20, activation='relu', name='dense2')(drop_layer1)
drop_layer2 = Dropout(rate=0.2, name='drop2')(dense_layer2)

dense_layer3 = Dense(units=1, activation='relu', name='dense3')(drop_layer2)

dft_model = Model(inputs=inputs, outputs=dense_layer3)

In [21]:
dft_model.compile(optimizer='sgd',
                 loss='mean_squared_error',
                 metrics=['acc'])

dft_model.fit(train_atom, train_en, verbose=1, 
              validation_data=[test_atom, test_en],
             epochs=2)

Train on 265167 samples, validate on 88389 samples
Epoch 1/2
265167/265167 [==============================] - 31s 118us/step - loss: 71878.5629 - acc: 0.9258 - val_loss: 8.6210e-05 - val_acc: 0.9262
Epoch 2/2
265167/265167 [==============================] - 28s 106us/step - loss: 1.0618e-04 - acc: 0.9258 - val_loss: 8.6210e-05 - val_acc: 0.9262
